In [7]:
from fastmcp import Client as MCPClient

from typing import Optional
from pprint import pp

# Add src directory to path for imports
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '../src'))
from helper import PROJECT_ROOT

## Get Tools from MCP server

In [8]:
MCP_URL = PROJECT_ROOT / "src/app_mcp.py"

In [9]:
async def list_tools_from_mcp(server_path: str):
    async with MCPClient(server_path) as mcp_client:
        tools = await mcp_client.list_tools()
    print("Available Tools:\n\n - " + '\n - '.join([tool.name for tool in tools]))

    return tools

In [10]:
async def generate_available_tools(mcp_url: str):
    """Generates JSON for available tools from MCP that can be passed directly to PG models"""
    tools = await list_tools_from_mcp(mcp_url)
    available_tools = []

    for tool in tools:
        tool_dict = {}
        tool_dict["type"] = "function"
        tool_dict["name"] = tool.name
        tool_dict["description"] = tool.description
        tool_dict["parameters"] = tool.inputSchema
        available_tools.append({"type": "function", "function": tool_dict, "strict": True})

    return available_tools

# Example usage:
AVAILABLE_TOOLS = await generate_available_tools(MCP_URL)

Available Tools:

 - analyze_script_punct
 - llm_chat


## Connect to LLM

In [6]:
from predictionguard import PredictionGuard
from dotenv import load_dotenv
import os
load_dotenv()

PREDICTIONGUARD_URL = os.getenv("PREDICTIONGUARD_URL", "https://api.predictionguard.com")
API_KEY = os.getenv("PREDICTIONGUARD_API_KEY")
MODEL = os.getenv("PREDICTIONGUARD_DEFAULT_MODEL", "gpt-oss-120b")
MAX_COMPLETION_TOKENS = 10000
TEMPERATURE = None

client = PredictionGuard(
    api_key=API_KEY,
    url=PREDICTIONGUARD_URL)

In [117]:
def initiate_chat(user_query: Optional[str] = None, file_name: Optional[str] = None):
    system_prompt = f"""You are an expert bible translator and consultant.\
You are responsible for analyzing translation tasks and provide accurate analysis and recommendations.\
You can either use the tools provided to you or use `llm_call` if you want to answer directly.\
Do not make up your own analysis, only use the tools provided."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    res = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=AVAILABLE_TOOLS,
            tool_choice="auto", 
            max_completion_tokens=MAX_COMPLETION_TOKENS,
            temperature=TEMPERATURE
        )
    
    return res

In [119]:
res = initiate_chat(user_query="Analyze the punctuation for this text: 'بنكتب لكم عن اللي كان من البداية بخصوص كلمة الحياة عن اللي'.")
res

{'id': 'chatcmpl-01eebe4597d34342b5e6946183ed5693',
 'object': 'chat.completion',
 'created': 1759829092,
 'model': 'OpenAI/gpt-oss-120b',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '',
    'tool_calls': [{'id': 'chatcmpl-tool-aa633d2e69c146d896737937183652a6',
      'type': 'function',
      'index': 0,
      'function': {'name': 'analyze_script_punct',
       'arguments': '{"input_filename": null, "input_string": "\\u0628\\u0646\\u0643\\u062a\\u0628 \\u0644\\u0643\\u0645 \\u0639\\u0646 \\u0627\\u0644\\u0644\\u064a \\u0643\\u0627\\u0646 \\u0645\\u0646 \\u0627\\u0644\\u0628\\u062f\\u0627\\u064a\\u0629 \\u0628\\u062e\\u0635\\u0648\\u0635 \\u0643\\u0644\\u0645\\u0629 \\u0627\\u0644\\u062d\\u064a\\u0627\\u0629 \\u0639\\u0646 \\u0627\\u0644\\u0644\\u064a", "lang_code": "ar", "lang_name": "Arabic"}'}}]},
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_tokens': 0,
  'completion_tokens_details': {'reasonin

In [ ]:
async def call_tool(mcp_url: str, tool_name: str, tool_args):
    """
    Calls the specified tool on the MCP server with the given arguments.

    Args:
        mcp_url (str): The MCP server URL.
        tool_name (str): The name of the tool to call.
        **tool_args: Arguments to pass to the tool.

    Returns:
        dict: The result of the tool call.
    """
    tool = next((t['function']['name'] for t in AVAILABLE_TOOLS if t['function']['name'] == tool_name), None)
    if not tool:
        raise ValueError(f"Tool '{tool_name}' not found on MCP server.")
   
    async with MCPClient(MCP_URL) as mcp_client:
        result = await mcp_client.call_tool(tool, tool_args)

    return result